In [ ]:
!pip install langchain openai pinecone-client chromadb PyMuPDF pdfplumber
!pip install langchain-community
!pip install pymupdf
!pip install openai
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.7 MB/s eta 0

In [ ]:

import fitz

def extract_text_from_pdf(pdf_file):
    try:
        doc = fitz.open(pdf_file)
        print("PDF opened successfully.")

        text = ""

        for page_num in range(doc.page_count):
            print(f"Extracting text from page {page_num + 1}")
            page = doc.load_page(page_num)
            text += page.get_text()
        doc.close()
        print("PDF closed successfully.")

        return text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


pdf_file = "/content/Project Report.docx"
extracted_text = extract_text_from_pdf(pdf_file)

if extracted_text:
    print("Extracted text from PDF:")
    print(extracted_text)
else:
    print("No text extracted.")

from langchain import OpenAI, ConversationChain

def generate_faqs(text):
    llm = OpenAI(temperature=0.7)
    conversation = ConversationChain(llm=llm)
    prompt = f"Extract frequently asked questions and answers from this text: {text}"
    faqs = conversation.run(prompt)
    return faqs

def generate_faqs(text):
    llm = OpenAI(temperature=0.7)
    conversation = ConversationChain(llm=llm)
    prompt = f"Extract frequently asked questions and answers from this text: {text}"
    faqs = conversation.run(prompt)
    return faqs
import openai
from pinecone import Pinecone, ServerlessSpec
openai.api_key = "sk-GjxoGFuBJKy0SVzY8Ffbw4OeuELJrLObBtiYcdZ-S3T3BlbkFJ0Nc6FvuEUL9gfTNRHqC_bbJo3Vf3QOi_QqIRTD3joA"
def create_embeddings(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    embedding = response['data'][0]['embedding']
    return embedding
pinecone_client = Pinecone(api_key="pcsk_YtPjB_AsNCKGMxvMov3J1ANU6pjfHBDe6AhcJdJMGVfDPAsVbhi9YyiLWrejkjFzWN8r6")

index_name = "faq-index"
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    index = pinecone_client.index(index_name)
def store_faq_in_pinecone(faqs):
    for faq in faqs:
        faq_embedding = create_embeddings(faq)
        index.upsert([(faq, faq_embedding)])
def search_faq(query):
    query_embedding = create_embeddings(query)
    results = index.query([query_embedding], top_k=5)
    return results

from langchain.chains import RetrievalQAWithSourcesChain

def enhanced_faq_response(query):
    relevant_faqs = search_faq(query)
    retriever = relevant_faqs.as_retriever()
    rag = RetrievalQAWithSourcesChain.from_chain_type(
        llm=OpenAI(),
        chain_type="stuff",
        retriever=retriever
    )

    response = rag(query)
    return response

PDF opened successfully.
Extracting text from page 1
Extracting text from page 2
Extracting text from page 3
Extracting text from page 4
Extracting text from page 5
Extracting text from page 6
Extracting text from page 7
Extracting text from page 8
Extracting text from page 9
Extracting text from page 10
PDF closed successfully.
Extracted text from PDF:
Team:
• Bhanu Prakash Munukutla -(Z23756866)
• Jaya Kumar Gontine-(Z23748721)
• Antony Alexia Sebasthikannu -(Z23721367)
• Sumanth Kotha-(Z23692910)
• Naga Mani Chappidi -(Z23755128)
• Yamini Reddy Pitta -(Z23744543)
Team:
• Bhanu Prakash Munukutla -(Z23756866)
• Jaya Kumar Gontine-(Z23748721)
• Antony Alexia Sebasthikannu -(Z23721367)
• Sumanth Kotha-(Z23692910)
• Naga Mani Chappidi -(Z23755128)
• Yamini Reddy Pitta -(Z23744543)
Farm to table
PROJECT REPORT
Farm to table
PROJECT REPORT
Abstract:
The proposed e-commerce platform revolutionizes the farm-to-table
concept by directly linking consumers with local organic farmers
and introdu